**Importing Libraries**

In [ ]:
import csv
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow import keras
 
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_poisson_deviance
from sklearn.metrics import mean_gamma_deviance

In [ ]:
#Reading File
data_file = 'BTCSTUSDT_1m_data.csv'
data = pd.read_csv(data_file)

#Setting up data
minute = pd.to_datetime(data['time']).dt.minute
hour = pd.to_datetime(data['time']).dt.hour
highPrice = pd.DataFrame(data['high'])
openPrice = pd.DataFrame(data['open'])
lowPrice = pd.DataFrame(data['low'])
closePrice = pd.DataFrame(data['close'])

finalData = pd.concat([openPrice, highPrice, lowPrice, closePrice, openPrice.shift(periods=-1)], axis=1)

highPrice = highPrice.shift(periods=-1)[:-2]
lowPrice = lowPrice.shift(periods=-1)[:-2]
finalData = finalData[:-2]

#Creating Nueral Network

dataShape = 5

input_layer = Input(shape=(dataShape), dtype='float32')
dense1 = Dense(60, activation='linear')(input_layer)
dense2 = Dense(60, activation='linear')(dense1)
dense3 = Dense(60, activation="linear")(dense2)
dense4 = Dense(60, activation="linear")(dense3)
dropout_layer = Dropout(0.1)(dense4)
output_layer = Dense(1, activation='linear')(dropout_layer)

#Compiling Models
denseModelHigh = Model(inputs=input_layer, outputs=output_layer)
denseModelHigh.compile(loss='mean_squared_error', optimizer='adam')
#denseModel.summary()

In [ ]:
#Dividing for High
finalDataSingle = finalData
lastLineLimit = finalDataSingle['low'].count()-1

size = finalDataSingle.columns.size
limit = size-1
y_high = highPrice.iloc[0:highPrice['high'].count()-1,:]
X_high = finalDataSingle.iloc[0:lastLineLimit, :]

X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(X_high, y_high, random_state=200, test_size=0.05)

X_train_high, X_valid_high, y_train_high, y_valid_high = train_test_split(X_train_high, y_train_high, test_size=0.05, random_state=1)

X_test_high = pd.concat([X_test_high, finalDataSingle.iloc[lastLineLimit:, :]])
y_test_high = pd.concat([y_test_high, highPrice.iloc[highPrice['high'].count()-1:, :]])

#Running Model:

denseModelHigh.fit(x=X_train_high, y=y_train_high, batch_size=50, epochs=500, verbose=1, validation_data=(X_valid_high, y_valid_high), shuffle=True)


Epoch 1/500
5/5 [==============================] - 1s 47ms/step - loss: 9.4193 - val_loss: 1.4675
Epoch 2/500
5/5 [==============================] - 0s 12ms/step - loss: 5.7865 - val_loss: 0.5670
Epoch 3/500
5/5 [==============================] - 0s 12ms/step - loss: 8.0486 - val_loss: 0.1528
Epoch 4/500
5/5 [==============================] - 0s 12ms/step - loss: 6.8289 - val_loss: 0.2685
Epoch 5/500
5/5 [==============================] - 0s 12ms/step - loss: 5.3340 - val_loss: 0.3742
Epoch 6/500
5/5 [==============================] - 0s 13ms/step - loss: 5.6265 - val_loss: 0.1382
Epoch 7/500
5/5 [==============================] - 0s 13ms/step - loss: 6.1795 - val_loss: 0.3385
Epoch 8/500
5/5 [==============================] - 0s 12ms/step - loss: 6.7595 - val_loss: 0.1599
Epoch 9/500
5/5 [==============================] - 0s 12ms/step - loss: 7.0609 - val_loss: 0.1378
Epoch 10/500
5/5 [==============================] - 0s 11ms/step - loss: 5.9368 - val_loss: 0.2299
Epoch 11/500
5/5 [=

In [ ]:
#Prediction
y_pred_high = denseModelHigh.predict(X_test_high)

r2score = r2_score(y_test_high, y_pred_high)
print('R-squared score for the test set:', round(r2score,4))

mse = mean_squared_error(y_test_high, y_pred_high)
print('Mear-squared Error for the test set:', mse)

mae = mean_absolute_error(y_test_high, y_pred_high)
print('Mean-Absolute Error for the test set:', mae)

print('Max Error: ', max_error(y_test_high, y_pred_high))

y_actual = pd.DataFrame(y_test_high, columns=['Actual High Price'])

y_hat = pd.DataFrame(y_pred_high, columns=['Predicted High Price'])

org_arr = np.array(y_test_high)
pre_arr = np.array(y_hat)

error = []
for i in range(0, y_actual.size):
  original = org_arr[i]
  predicted = pre_arr[i]
  error.append(((original-predicted)/original)*100)
  #print(original, ": ", predicted, ", Error: ", error[i])

error.sort(reverse=True)

errorDF = pd.DataFrame(error)

print(errorDF.describe())
print(org_arr[y_actual.size-1], " - ", pre_arr[y_actual.size-1])


R-squared score for the test set: 0.9634
Mear-squared Error for the test set: 1.1196771284541471
Mean-Absolute Error for the test set: 1.0046473795572917
Max Error:  1.6584663391113281
               0
count  15.000000
mean    1.784767
std     0.646349
min     0.885948
25%     1.248832
50%     1.566301
75%     2.244370
max     2.961547
[66.286]  -  [65.51861]


In [ ]:
filename = '/content/drive/MyDrive/MyModel/highPriceModel'
 
denseModelHigh = keras.models.load_model(filename)

In [ ]:
#OPen Testing
newOpenValue = 65.599
openValue = 64.833
highValue = 65.913
lowValue = 64.798
closeValue = 65.699
newData = pd.DataFrame([[openValue, highValue, lowValue, closeValue, newOpenValue]])
#, columns = X_test_high.columns)
 
newPredictionHigh = denseModelHigh.predict(newData)
 
print('High: ', newPredictionHigh[0][0])

High:  64.84176


In [ ]:
filename = '/content/drive/MyDrive/MyModel/highPriceModel'

denseModelHigh.save(filename)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/MyModel/highPriceModel/assets
